## Quantum chemistry simulation using the Variational Quantum Eigensolver 

Note that I needed to install a minor version of h5py (PySCF does not support h5py >= 3)

pip install h5py==2.10.0 qiskit qiskit-nature pyscf

### 1. Basic VQE calculation

Import the basic libraries

In [ ]:
import numpy as np
# Import standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.providers.aer import StatevectorSimulator
from qiskit.utils import QuantumInstance
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.ibmq import least_busy

Import the Qiskit Nature libraries for quantum chemistry simulation

In [ ]:
# Import Qiskit libraries for VQE
from qiskit.algorithms import NumPyMinimumEigensolver, VQE
from qiskit.algorithms.optimizers import COBYLA, SLSQP, L_BFGS_B, SPSA
# Import Qiskit Nature libraries
from qiskit_nature.drivers import PySCFDriver, UnitsType, Molecule
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper
from qiskit_nature.transformers import ActiveSpaceTransformer
from qiskit_nature.algorithms import VQEUCCFactory, GroundStateEigensolver
from qiskit_nature.circuit.library import HartreeFock, UCCSD, PUCCD

Setup the molecular data for simulation
Note that this step requires an external program (this example uses the PySCF program) for Hartree-Fock and integral calculations

Some typical input parameters for quantum chemistry simulation
(Note that the conventions differ among programs used)
1. Cartesian coordinates with atomic labels (Positions of the nuclei)
2. Charge (-1: anion, +1: cation, 0: neutral molecule)
3. Multiplicity (1:singlet, 2:doublet, 3:triplet, .....)
4. Basis set (sto-3g), minao, 6-31g, cc-pvdz, ....)

In [ ]:
# Setup the molecule
molecule = Molecule(geometry=[['H', [0., 0., 0.]],
                              ['H', [0., 0., 0.735]]],
                     charge=0, multiplicity=1)
# Initiate the PySCF driver (for setup of quantum chemical calculations and Hartee-Fock calculation)
driver = PySCFDriver(molecule = molecule, unit=UnitsType.ANGSTROM, basis='sto-3g')

Perform the calculation using PySCF and construct the second quantizied Hamiltonian

In [ ]:
# Perform the setup calculation using PySCF and retrieve data from the calculations
es_problem = ElectronicStructureProblem(driver)
# Get the second quantized Hamiltonian
second_q_op = es_problem.second_q_ops()
print(second_q_op[0])

Transform the Hamiltonian into qubitized form (Jordan-Wigner mapping)

In [ ]:
qubit_converter = QubitConverter(mapper=JordanWignerMapper())
qubit_op = qubit_converter.convert(second_q_op[0])
print(qubit_op)

Comparison with two-qubit reduction (Parity mapping)

In [ ]:
qubit_converter2 = QubitConverter(mapper = ParityMapper(), two_qubit_reduction=True)
qubit_op2 = qubit_converter2.convert(second_q_op[0], num_particles=es_problem.num_particles)
print(qubit_op2)

Set the solver for the reference value

In [ ]:
# Set the solver for the exact value
numpy_solver = NumPyMinimumEigensolver()

Perform reference calculation

In [ ]:
calc = GroundStateEigensolver(qubit_converter, numpy_solver)
res = calc.solve(es_problem)
print(res)

Setup for the VQE calculation

In [ ]:
# Set the optimizer (COBYLA, L_BFGS_B (L-BFGS-B), and etc.)  
# See https://github.com/Qiskit/qiskit-terra/tree/main/qiskit/algorithms/optimizers
optimizer = SLSQP(maxiter=100) 

# Set number of spin-orbitals (If you change the basis set, you may need to change this parameter as well)
num_spin_orbitals=4
# Set the number of particles
num_particles=(1,1)

# Set the initial state as the Hartree-Fock state | 1100 > or | 01 01 > in Qiskit convention
initial_state = HartreeFock(num_spin_orbitals, num_particles, qubit_converter)

# Set the UCCSD ansatz
ansatz = UCCSD(qubit_converter,num_particles,num_spin_orbitals,initial_state=initial_state,)

# Set the VQE solver
vqe_solver = VQE(ansatz=ansatz,optimizer=optimizer,
            quantum_instance=QuantumInstance(backend=Aer.get_backend("statevector_simulator")),)

Perform the VQE calculation using Ground state eigensolver

In [ ]:
from qiskit_nature.algorithms import GroundStateEigensolver
# Perform the VQE calculation using the Ground state eigensolver
calc = GroundStateEigensolver(qubit_converter, vqe_solver)
res = calc.solve(es_problem)
print(res)

### 2. Construct a potential energy curve

Calculate the molecular energy while elongating the H-H bond length.

In [ ]:
h2_length = np.arange(0.5, 2.5, 0.1)
hf_energy_list=[]
uccsd_energy_list=[]
for bond_length in h2_length:
    # This is simpler than the above example
    molecule = Molecule(geometry=[['H', [0., 0., 0.]],
                                  ['H', [0., 0., bond_length]]],
                         charge=0, multiplicity=1)
    driver = PySCFDriver(molecule = molecule, unit=UnitsType.ANGSTROM, basis='sto6g')
    es_problem = ElectronicStructureProblem(driver)
    qubit_converter = QubitConverter(mapper = ParityMapper(), two_qubit_reduction=True)
    quantum_instance = QuantumInstance(backend = Aer.get_backend('statevector_simulator'))
    vqe_solver = VQEUCCFactory(quantum_instance)
    calc = GroundStateEigensolver(qubit_converter, vqe_solver)
    res = calc.solve(es_problem)
    hf_energy_list += [res.hartree_fock_energy] 
    uccsd_energy_list += [res.total_energies[0]] 

Plot the energies (Confirm that UCCSD will converge at the dissociation limit, while Hartree-Fock does not)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(h2_length,hf_energy_list,color='black',label='HF')
plt.plot(h2_length,uccsd_energy_list,color='red',label='UCCSD')
plt.title("Potential energy curve of hydrogen molecule")
plt.xlabel("H-H bond length")
plt.ylabel("Energy (hartrees)")
plt.legend()

### 3. Reduce the problem size

### Reference (Full size problem (Takes about 5-10 minutes))

In [ ]:
# define molecule
molecule = Molecule(geometry=[['Li', [0., 0., 0.]],
                              ['H',  [0., 0., 2.5]]],
                        charge=0, 
                        multiplicity=1)

# specify driver
driver = PySCFDriver(molecule=molecule, unit=UnitsType.ANGSTROM, basis='sto3g')

# define electronic structure problem
problem = ElectronicStructureProblem(driver)

# construct qubit converter (Jordan-Wigner)
qubit_converter = QubitConverter(mapper=JordanWignerMapper())

# Set the optimizer (COBYLA, L_BFGS_B (L-BFGS-B), and etc.)  
# See https://github.com/Qiskit/qiskit-terra/tree/main/qiskit/algorithms/optimizers
optimizer = SLSQP(maxiter=100) 

# Set number of spin-orbitals
num_spin_orbitals=12
# Set the number of particles
num_particles=(2,2)

# Set the initial state as the Hartree-Fock state | 1100 > or | 01 01 > in Qiskit convention
initial_state = HartreeFock(num_spin_orbitals, num_particles, qubit_converter)

# Set the UCCSD ansatz
ansatz = UCCSD(qubit_converter=qubit_converter,num_particles=num_particles,num_spin_orbitals=num_spin_orbitals,initial_state=initial_state,)

# Set the VQE solver
vqe_solver = VQE(ansatz=ansatz,optimizer=optimizer,
            quantum_instance=QuantumInstance(backend=Aer.get_backend("statevector_simulator")),)

calc = GroundStateEigensolver(qubit_converter, vqe_solver)
res_UCCSD = calc.solve(problem)
lih_uccsd_energy= res_UCCSD.total_energies[0]
lih_hf_energy = res_UCCSD.hartree_fock_energy
print('LiH HF energy = ', lih_hf_energy)
print('LiH UCCSD energy = ', lih_uccsd_energy)

Calculate the reference energy

In [ ]:
np_solver = NumPyMinimumEigensolver()
np_groundstate_solver = GroundStateEigensolver(qubit_converter, np_solver)

np_result = np_groundstate_solver.solve(problem)

target_energy = np.real(np_result.eigenenergies + np_result.nuclear_repulsion_energy)[0]
print('LiH exact energy:', target_energy)

### Change the ansatz

In [ ]:
# define electronic structure problem
problem = ElectronicStructureProblem(driver)

# construct qubit converter (Jordan-Wigner)
qubit_converter = QubitConverter(mapper=JordanWignerMapper())

# Set the optimizer (COBYLA, L_BFGS_B (L-BFGS-B), and etc.)  
# See https://github.com/Qiskit/qiskit-terra/tree/main/qiskit/algorithms/optimizers
optimizer = SLSQP(maxiter=100) 

# Set number of spin-orbitals
num_spin_orbitals=12
# Set the number of particles
num_particles=(2,2)

# Set the initial state as the Hartree-Fock state | 1100 > or | 01 01 > in Qiskit convention
initial_state = HartreeFock(num_spin_orbitals, num_particles, qubit_converter)

# Set the PUCCD ansatz (You can also try changing PUCCD to SUCCD)
ansatz = PUCCD(qubit_converter=qubit_converter,num_particles=num_particles,num_spin_orbitals=num_spin_orbitals,initial_state=initial_state,)

# Set the VQE solver
vqe_solver = VQE(ansatz=ansatz,optimizer=optimizer,
            quantum_instance=QuantumInstance(backend=Aer.get_backend("statevector_simulator")),)

calc = GroundStateEigensolver(qubit_converter, vqe_solver)
res_PUCCD = calc.solve(problem)
lih_puccd_energy= res_PUCCD.total_energies[0]
print('LiH PUCCD energy = ', lih_puccd_energy)

### Reduce the active space

In [ ]:
# Run PySCF to get the parameters for the molecule
q_molecule = driver.run()
    
# Specify active space transformation (You can also try changing the number of orbitals)
active_space_trafo = ActiveSpaceTransformer(num_electrons=(q_molecule.num_alpha, q_molecule.num_beta),
                                            num_molecular_orbitals=3)
    
# define electronic structure problem
problem = ElectronicStructureProblem(driver, q_molecule_transformers=[active_space_trafo])

# construct qubit converter (parity mapping + 2-qubit reduction)
qubit_converter = QubitConverter(mapper=JordanWignerMapper())

# If you change num_molecular_orbitals, set num_spin_orbitals = num_molecular_orbitals*2 
num_spin_orbitals = 6
num_particles=(2,2)

# Set the initial state as the Hartree-Fock state | 1100 > or | 01 01 > in Qiskit convention
initial_state = HartreeFock(num_spin_orbitals, num_particles, qubit_converter)

# Set the UCCSD ansatz
ansatz = UCCSD(qubit_converter=qubit_converter,num_particles=num_particles,num_spin_orbitals=num_spin_orbitals,initial_state=initial_state,)

# Set the VQE solver
vqe_solver = VQE(ansatz=ansatz,optimizer=optimizer,
            quantum_instance=QuantumInstance(backend=Aer.get_backend("statevector_simulator")),)

calc = GroundStateEigensolver(qubit_converter, vqe_solver)
res_UCCSD_active = calc.solve(problem)

lih_uccsd_energy_active= res_UCCSD_active.total_energies[0]
print('LiH UCCSD energy with active space = ', lih_uccsd_energy_active)

Compare the results of all LiH simulations. Investigate the performance of the problem size reduction. 

In [ ]:
print('LiH HF energy = ', lih_hf_energy)
print('LiH UCCSD energy with active space = ', lih_uccsd_energy_active)
print('LiH PUCCD energy = ', lih_puccd_energy)
print('LiH UCCSD energy = ', lih_uccsd_energy)
print('LiH exact energy:', target_energy)

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright